## Flights and Domestic Airlines 
Providing the best recommendations for domestic travel in the U.S. In our general applications we are using 2015 data to determine delay information and predict which airlines contains the best flights. 

We are also determining if the weather that day was the reason for the flight cancellation or weather the airline was at fault.

Finally with flight. We are looking into Hotel Recommendations and determining the best hotels domestically based on reviews.


In [1]:
# Install mysql connector, SQLAlchemy
import pandas as pd; import numpy as np, os, mysql.connector as maria; from sqlalchemy import create_engine
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Time, DateTime, Date 

In [62]:
from datetime import datetime
date_data  ='0005' 
data = datetime.strptime(date_data, '%H%M')
result = data.strftime('%H:%M:%S')
print(result)

00:05:00


In [84]:
# Get Flight Delay information and cleanse the data for missing values

# Airports CSV
airport_info = pd.read_csv("airports.csv", sep = ",")

# Airlines CSV
airlines_info = pd.read_csv("airlines.csv", sep = ",")
airlines_info.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [85]:
airport_info.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [132]:
import pandas as pd
# Flight information (5,819,079  Rows)
# First chunk of a million rows
my_chunk = pd.read_csv("flights.csv", sep=',', nrows =1000, dtype={'SCHEDULED_DEPARTURE': str,'DEPARTURE_TIME':str, 'WHEELS_OFF': str,
'WHEELS_ON': str,'SCHEDULED_ARRIVAL': str,'ARRIVAL_TIME':str}) # Read Rows in Smaller Numbers or chunks and update the time columns with readable text

columns = list(my_chunk) # Create a list of column names for the dataset

my_chunk.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,0005,...,0408,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,0010,...,0741,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,0020,...,0811,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,0020,...,0756,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,0025,...,0259,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
# Cleanse Data

try:
    # Concatenate Date columns and give day of the week (e.g. 1/1/2015 and Sunday instead of 7)
    my_chunk.insert(loc=0, column = "DATE", value =my_chunk["MONTH"].map(str) + "/" + my_chunk['DAY'].map(str) + "/" + my_chunk['YEAR'].map(str))

    # Drop individual dates from columns
    my_chunk = my_chunk.drop(columns = ['YEAR','MONTH', 'DAY'])


    #Map days of the week to numeric values
    days_of_week = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7:'Sunday'}
    my_chunk["DAY_OF_WEEK"].replace(days_of_week, inplace=True) # match category names with values in the dictionary
    columns = list(my_chunk)
    # Change the times in the dataframe so they are formatted as HH:MM (e.g. 23:54 instead of 2354)
    time_index = [7,8,11,16,18,19] # values in the row that needs time adjustments based on manual effort

    for t in time_index:
        my_chunk[columns[t]] = pd.to_datetime(my_chunk[columns[t]], format='%H%M')
        my_chunk[columns[t]] = my_chunk[columns[t]].dt.time
except:
    print("Data has already been cleansed")
    
for t in time_index:
    print(my_chunk[columns[t]].head())

0    00:05:00
1    00:10:00
2    00:20:00
3    00:20:00
4    00:25:00
Name: SCHEDULED_DEPARTURE, dtype: object
0    23:54:00
1    00:02:00
2    00:18:00
3    00:15:00
4    00:24:00
Name: DEPARTURE_TIME, dtype: object
0    00:15:00
1    00:14:00
2    00:34:00
3    00:30:00
4    00:35:00
Name: WHEELS_OFF, dtype: object
0    04:04:00
1    07:37:00
2    08:00:00
3    07:48:00
4    02:54:00
Name: WHEELS_ON, dtype: object
0    04:30:00
1    07:50:00
2    08:06:00
3    08:05:00
4    03:20:00
Name: SCHEDULED_ARRIVAL, dtype: object
0    04:08:00
1    07:41:00
2    08:11:00
3    07:56:00
4    02:59:00
Name: ARRIVAL_TIME, dtype: object


In [29]:
# Data Storage
# Create Table that will store flight data in MariaDB or MySQL


meta = MetaData()

# Run this cell once
try:
    flights = Table(
       'flights', meta, 
       Column('YEAR', Integer)
    )

    meta.create_all(engine)

    def add_column(engine, table_name, column):
        column_name = column.compile(dialect=engine.dialect)
        column_type = column.type.compile(engine.dialect)
        engine.execute('ALTER TABLE %s ADD COLUMN %s %s' % (table_name, column_name, column_type))
    
    dtypes = [Date, String(50), String(50), String(50), String(6),String(3),String(3), Time, Time, Integer
              , Integer, Time, Integer, Integer, Integer, Integer, Time, Integer, Time, Time, Integer, String(50)
              , String(50), String(1), Integer, Integer, Integer, Integer, Integer]
   
    columns = list(my_chunk)
    for c,d in zip(columns,dtypes):
        column = Column(c,d)
        add_column(engine, "flights", column)
except:
    print("The Database already has the columns!")

try:
    engine.execute('ALTER TABLE %s DROP COLUMN %s' % ('flights', 'YEAR'))
except:
    print("Column has already been dropped")

# Import the first 1,000,000 rows into database for further analysis
#my_chunk.to_sql(name='flights', con=engine, if_exists = 'append', index=False)


In [ ]:
# Create Machine Learning Model to predict if Airline probability of having a delay (SciKit Learn Values)

In [ ]:
# Use Weather API to see if weather affected the flight status (e.g. Weatherbug)

In [ ]:
# Create Visualizations of Airport information